In [2]:
import cv2
import mediapipe as mp
import numpy as np
import random
import time

# Inicializa MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# Inicializa câmera
cap = cv2.VideoCapture(0)

# Tabuleiro do jogo (3x3)
board = [["" for _ in range(3)] for _ in range(3)]
vez_jogador = True
estado_jogo = "inicio"  # Começa na tela inicial
vencedor = ""
tempo_fim = 0

# Tempo para a contagem regressiva da tela inicial
tempo_contagem = 3  # segundos
inicio_contagem = time.time()

# Função para desenhar o tabuleiro
def desenhar_tabuleiro(img, board):
    h, w, _ = img.shape
    step_x = w // 3
    step_y = h // 3

    # Linhas do tabuleiro
    for i in range(1, 3):
        cv2.line(img, (0, i * step_y), (w, i * step_y), (255, 255, 255), 2)
        cv2.line(img, (i * step_x, 0), (i * step_x, h), (255, 255, 255), 2)

    # Símbolos X e O
    for i in range(3):
        for j in range(3):
            cx = j * step_x + step_x // 2
            cy = i * step_y + step_y // 2
            if board[i][j] == "X":
                cv2.putText(img, "X", (cx - 20, cy + 20), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)
            elif board[i][j] == "O":
                cv2.putText(img, "O", (cx - 20, cy + 20), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)

# Verifica se há um vencedor
def verificar_vitoria(b):
    # Linhas, colunas e diagonais
    for i in range(3):
        if b[i][0] == b[i][1] == b[i][2] != "":
            return b[i][0]
        if b[0][i] == b[1][i] == b[2][i] != "":
            return b[0][i]
    if b[0][0] == b[1][1] == b[2][2] != "":
        return b[0][0]
    if b[0][2] == b[1][1] == b[2][0] != "":
        return b[0][2]
    return ""

# Verifica empate
def verificar_empate(b):
    for row in b:
        if "" in row:
            return False
    return True

# Retorna a posição da ponta do dedo indicador
def obter_posicao_dedo_indicador(landmarks, w, h):
    x = int(landmarks[8].x * w)
    y = int(landmarks[8].y * h)
    return x, y

# CPU faz uma jogada aleatória
def jogada_maquina():
    vazios = [(i, j) for i in range(3) for j in range(3) if board[i][j] == ""]
    if vazios:
        i, j = random.choice(vazios)
        board[i][j] = "O"

# Loop principal
while True:
    success, img = cap.read()
    if not success:
        break

    img = cv2.flip(img, 1)
    h, w, _ = img.shape

    if estado_jogo == "inicio":
        # Tela preta
        img[:] = (0, 0, 0)
        tempo_restante = int(tempo_contagem - (time.time() - inicio_contagem)) + 1
        if tempo_restante > 0:
            cv2.putText(img, "Prepare-se!", (120, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 4)
            cv2.putText(img, str(tempo_restante), (280, 280), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5)
        else:
            estado_jogo = "jogando"
            vez_jogador = True
            continue

    elif estado_jogo == "jogando":
        desenhar_tabuleiro(img, board)

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)

        if results.multi_hand_landmarks and vez_jogador:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                x, y = obter_posicao_dedo_indicador(hand_landmarks.landmark, w, h)
                cv2.circle(img, (x, y), 10, (255, 0, 0), -1)

                col = x // (w // 3)
                row = y // (h // 3)

                if 0 <= row <= 2 and 0 <= col <= 2:
                    if board[row][col] == "":
                        board[row][col] = "X"
                        vez_jogador = False
                        break

        if not vez_jogador:
            time.sleep(0.5)
            jogada_maquina()
            vez_jogador = True

        vencedor = verificar_vitoria(board)
        if vencedor != "":
            estado_jogo = "fim"
            tempo_fim = time.time()
        elif verificar_empate(board):
            estado_jogo = "empate"
            tempo_fim = time.time()

    # Exibe resultado se houver
    if estado_jogo == "fim":
        cv2.putText(img, f"{vencedor} venceu!", (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 4)
    elif estado_jogo == "empate":
        cv2.putText(img, "Empate!", (200, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 4)

    # Reinicia após 3 segundos
    if estado_jogo in ["fim", "empate"] and (time.time() - tempo_fim > 3):
        board = [["" for _ in range(3)] for _ in range(3)]
        estado_jogo = "inicio"
        inicio_contagem = time.time()
        vencedor = ""

    cv2.imshow("Jogo da Velha", img)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
